In [1]:
from bs4 import BeautifulSoup as bs
import requests
import time
import re
from datetime import datetime,timedelta

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

일반청원은 공개되지 않으며  
공개청원의 경우  
접수>공개여부 결정(15일)>국민의견 수렴(30일)>조사,심의>결과통지(90일내)  
의 과정을 거침

title:청원 제목  
detail:세부내용  
date:올라온 날짜(의견수렴기간 -15일)
period:의견 수렴 기간    *필요한가?* 
state:접수/의견수렴중/처리중/종결 네가지로 나뉘며 진행중인 상태의 태그에 class="on"  

views:조회수  
comments:댓글수, + 일반적으로 의견수렴을 댓글로 하는듯 *댓글 갯수만 가져왔는데, 댓글내용도 필요?*  

result:처리 내용
agency:처리 기관 *추가됨*  
agency_detail : 담당 기관 세부 부서
post_url: 파라미터를 넣은 게시글 주소 *추가됨*   

In [2]:
petition_dict={"title":[],
               "detail":[],
               "date":[],
               "period":[],
               "state":[],
               "views":[],
               "comment_num":[],
               "result":[],
               "agency":[],
               "agency_detail":[],
               "post_url":[]
              }

# 게시글 리스트에서 확인가능(request)

In [21]:
res=requests.get("https://cheongwon.go.kr/portal/petition/open/view?pageIndex=1&searchType=1&searchKeyword=&type=list")
soup=bs(res.text,"html.parser")

## title

In [22]:
title_tags=soup.find_all("span","subject")
for title in title_tags:
    petition_dict["title"].append( title.get("title"))
    
print(petition_dict["title"])
    

['내일배움카드 제도 개선', '가족이 원수다.~~~~^^', '온수 급탕비 & 난반비 인하 시켜주세요!', '대통령에게 바란다 - 한국에 필요한 3가지', '산업스파이 처벌강화를 요구합니다', '유아교육법 개정', '폐결핵', '산재보험의 잘못된 법으로 인하여 국민정서를 해치고 있다', 'SH청년안심주택 유자녀 주차 나이 제한(만 6세미만) 조항 이의제기합니다.']


## period

In [58]:
period_tags=soup.find_all("span","date")
for period in period_tags:
    #날짜만 구함
    period=period.get_text()
    period=period.split(':')[1].strip()

    #날짜형식으로 바꾸기
    period = period.split('~')
    period[0]=datetime.strptime(period[0],'%Y.%m.%d')
    period[1]=datetime.strptime(period[1],'%Y.%m.%d')
    
    petition_dict["period"].append(period)
    
print(petition_dict["period"])
    

[[datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)], [datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)], [datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)], [datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)], [datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)], [datetime.datetime(2024, 4, 15, 0, 0), datetime.datetime(2024, 5, 14, 0, 0)], [datetime.datetime(2024, 4, 13, 0, 0), datetime.datetime(2024, 5, 13, 0, 0)], [datetime.datetime(2024, 4, 13, 0, 0), datetime.datetime(2024, 5, 13, 0, 0)], [datetime.datetime(2024, 4, 13, 0, 0), datetime.datetime(2024, 5, 13, 0, 0)]]


## date

In [61]:
for date in petition_dict["period"]:
    date=date[0]-timedelta(days=15)
    petition_dict["date"].append(date)
    
print(petition_dict["date"])

[datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)]
[datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)]
[datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)]
[datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)]
[datetime.datetime(2024, 4, 16, 0, 0), datetime.datetime(2024, 5, 16, 0, 0)]
[datetime.datetime(2024, 4, 15, 0, 0), datetime.datetime(2024, 5, 14, 0, 0)]
[datetime.datetime(2024, 4, 13, 0, 0), datetime.datetime(2024, 5, 13, 0, 0)]
[datetime.datetime(2024, 4, 13, 0, 0), datetime.datetime(2024, 5, 13, 0, 0)]
[datetime.datetime(2024, 4, 13, 0, 0), datetime.datetime(2024, 5, 13, 0, 0)]
[datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 3, 31, 0, 0), datetime.datetime(2024, 3, 29, 0, 0), datetime.datetime(2024, 3, 29, 0, 0), datetim

## url

In [50]:
post_id=soup.find_all("a", onclick=True)
for post in post_id :
    if post.get("style") == "cursor: pointer;":
        post=post.get("onclick").split('\'')[1]
        print(post)


PRI80995be2cefd4f3fa4b9fb4b6f1204c8
PRI1883cf28daec49489233a34f6e169fcc
PRI93490eaeb1f542c7a25e7053ebff5e23
PRIa69b2e5c84fc4bb7b48d0a3f46889b50
PRI7fd9b46c718042df900fd074815f07e9
PRIbacec5b69a41436e892ccc91de3dec8b
PRIe5987e1356dd489bb1bc50b61db9d041
PRIa237df283734430dbe8e0e5d77efd65d
PRI97357c6b00284def95c8c43eaa2ebe24


# 게시글에서 확인가능(requests+selenium(view))

In [64]:
res2=requests.get("https://cheongwon.go.kr/portal/petition/open/viewdetail/PRIea74755c07fc4a0fbb178fdedb23cf6b")
soup2=bs(res2.text,"html.parser")

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://cheongwon.go.kr/portal/petition/open/viewdetail/PRI89cf07387ec84a0fb943a01651040e8b")


## detail

In [27]:
details=soup2.find("div","pet-doc__cont")
details=details.text

#\n이랑 쓸데없는 공백 &nbsp처리
details=cleaned_text = re.sub(r'\s+', ' ', details)
print(details)

금융권 은행들 대출이자 금리장사해서 역대급 성과급 잔치... 당장 최저 금리로 인하 시켜주세요! 


## result
있는 경우

In [28]:
details=soup2.find("div","pet-doc__result")
details=details.text

#\n이랑 쓸데없는 공백 &nbsp처리
details=cleaned_text = re.sub(r'\s+', ' ', details)
print(details)

 <청원 처리결과> □ '23.11.15.에 접수된 귀하의 청원(20231101-1103000-0002)에 대한 회신입니다. □ 귀하의 관심과 제언에 감사드립니다. 다만, 은행의 대출 금리는 조달비용, 신용원가 등을 감안하여 개별 은행이 자율적으로 결정하는 사안에 해당하므로, 원칙적으로 우리원이 간여하기 어려움을 알려드립니다. □ 다만, 귀하께서 말씀하신 내용은 향후 우리원 업무추진시 참고하여 은행의 금융서비스가 보다 금융소비자 친화적으로 이루어질 수 있도록 노력하겠습니다. 끝. 


## state

In [29]:
state=soup2.find("ul","process").find('li','on')
state=state.text
state

'종결'

## views
딱히 태그가 없어 보여서...selenium xpath로 검색

In [36]:
views=driver.find_element(By.XPATH,'//*[@id="lay-content"]/div/div/div[1]/div[1]/div[2]/div').text
views=views.split('조회 수 ')
views=views[1]
views

'27'

## comment_num

In [33]:
comment_num=soup2.find("div","total").find('span','blue')
comment_num=comment_num.text
comment_num

'5'

# 순회

### 질문
>태그가 없는 항목("View") 때문에 selenium이 필요하긴한데...
>브라우저 세션을 여는 것때문에 일반적으로 request가 서버 부하도 덜하대서 고민중


>그거 말고도 selenium으로 게시판 아래 >>버튼 눌러서
>
> 클릭해서 끝페이지 부터 읽어올지/아니면 그냥 순회 번호 입력할지 고민중
>
> bs4로 onclick속성이 있는 a태그를 읽었을때 마지막 태그가 >>라서 
>
>  `<a class="cs-paging__last" href="?pageIndex=218" onclick="linkPage(218);return false; ">끝 페이지</a>]`
>
>  에서 linkpage안의 값 읽어오면 되긴하는데... 옛날것부터 넣을거임? 그럼 url 에 searchType=3으로 바꾸면 됨



>그리고 코드가 너무...난잡한거같아서 고민
>자식관계 생각안하고 그냥 class로 접근 가능해서 그렇게 했는데
>동일한 부모를 가진 태그를 뽑고 그 태그 내에서 동일한 값들을 뽑는게 더 깔끔한지?

>게시글 접근 시 오류가 뜨는데 코드의 문제는 아닌거 같음, 대기 시간을 줘야하는지?


### selenium(아직 안함)

In [3]:
for i in range(1,10):#게시판 순회
    driver.get("https://cheongwon.go.kr/portal/petition/open/view?pageIndex=1&searchType=1&searchKeyword=&type=list")
    WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//body')))
    
    #게시글 주소
    id_list = driver.find_elements(By.XPATH,"//a[contains(@onclick, 'fn_detailView')]")
    for id in id_list:
        id=id.get_attribute("onclick").split('\'')[1]
    
    


NameError: name 'driver' is not defined

### requests

In [4]:
header={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}


res=requests.get("https://cheongwon.go.kr/portal/petition/open/view",headers=header)
soup=bs(res.text,"html.parser")

In [5]:
final_index=soup.find('a','cs-paging__last').get('href')
final_index=int(final_index.split("=")[-1])
final_index

218

In [7]:
#page_index번쨰 페이지  post_count번째 글
#page_index*9+post_count-1번째 글
#page_index*9+post_count-2 가 dict 위치
for page_index in range(1, final_index):
    res=requests.get(f"https://cheongwon.go.kr/portal/petition/open/view?pageIndex={page_index}&searchType=1&searchKeyword=&type=list",headers=header)
    soup=bs(res.text,"html.parser")
    post_count=0

    #게시판에서 얻을 수 있는 정보들
    #url
    #period
    #date
    #title
    #agency


    ###각 게시글들 url구하기(url)
    #https://cheongwon.go.kr/portal/petition/open/viewdetail/{추출한 속성}
    thispage_posturl=[]#해당 페이지 게시글 탐색시 사용할 url리스트
    a_tag=soup.find_all("a", onclick=True)
    for post in a_tag : #a_tag중 게시글 태그만
        if post.get("style") == "cursor: pointer;":
            post=post.get("onclick").split('\'')[1]
            post_url="https://cheongwon.go.kr/portal/petition/open/viewdetail/"+post
            # petition_dict["post"].append(post)
            petition_dict["post_url"].append(post_url)
            thispage_posturl.append("\""+post_url+"\"")




    ###청원 의견 수렴(동의)기간 구하기(period)
    ###청원 신청 날짜 구하기 == 의견수렴 날짜 -15(date)
    period_tags=soup.find_all("span","date")
    for period in period_tags:
        #날짜만 구함
        period=period.get_text()
        period=period.split(':')[1].strip()
        #[시작 날짜, 종료 날짜]
        period = period.split('~')
        #형식 바꿔서 넣을 period_delta
        period_delta=[]
        #datetime형식으로 바꾸기 
        period_delta.append(datetime.strptime(period[0],'%Y.%m.%d'))
        period_delta.append(datetime.strptime(period[1],'%Y.%m.%d'))
        #datetime형식으로 넣기 싫으면 period넣으면됨
        petition_dict["period"].append(period_delta)
        
        date=period_delta[0]-timedelta(days=15)
        #datetime형식으로 넣기 싫으면 아래 코드 주석 처리 해제
        #date=date.strftime("%Y.%m.%d")
        petition_dict["date"].append(date)

    ###청원 제목 구하기(title)
    title_tags=soup.find_all("span","subject")
    for title in title_tags:
        petition_dict["title"].append( title.get("title"))

    ###담당부서 구하기(agency)
    agency_tags=soup.find_all("span","category")
    for agency in agency_tags:
        petition_dict["agency"].append(agency.text)
        
    #게시글에서 추가로 얻을 수 있는 정보들
    #details
    #result
    #state
    #view
    #comment_num     근데 이거 반대는 없지만 댓글로 반대의견 표현하는 경우도 있어서 고민중... 일단 comment num만 받음
    time.sleep(2)
    WebDriverWait
    for url in thispage_posturl: #해당 페이지의 게시글 url이 들어있음
            post_count+=1
            res_=requests.get(url,headers=header)
            soup_=bs(res_.text,"html.parser")

            ###청원 내용 구하기(detail)
            detail=soup_.find("div","pet-doc__cont")
            detail=details.text
            #\n이랑 쓸데없는 공백 &nbsp처리
            detail=cleaned_text = re.sub(r'\s+', ' ', details)
            petition_dict["detail"].append(detail)

            ### 처리 결과 구하기  (result)
            result=soup_.find("div","pet-doc__result")
            if result : #답변이 존재할 때
                result=result.text
                #\n이랑 쓸데없는 공백 &nbsp처리
                result = re.sub(r'\s+', ' ', result)
                petition_dict["result"].append(result)
            else : #답변이 없을 때>None삽입
                petition_dict["result"].append(None)
                
            ### 상태 구하기(state)
            state=soup_.find("ul","process").find('li','on')
            state=state.text
            petition_dict["state"].append(state)

            ### 댓글 숫자 구하기(comment_num)
            comment_num=soup_.find("div","total").find('span','blue')
            comment_num=comment_num.text
            petition_dict["comment_num"].append(comment_num)

            ###조회수 구하기 (views)
            views = soup_.find("div","link").find("div")
            views=views.text.replace('조회 수 ','').strip()
            petition_dict['views'].append(views)

            ### 세부담당 부서 구하기(agency_detail)
            agency_detail=soup_.find("div","category").find("span")
            agency_detail=agency_detail.text.replace("처리기관: ","").strip()
            petition_dict['agency_detail']=agency_detail
            time.sleep(2)

InvalidSchema: No connection adapters were found for '"https://cheongwon.go.kr/portal/petition/open/viewdetail/PRI4d82f08dedcd4af58b575527a3b5ef58"'

In [108]:
x="https://cheongwon.go.kr/portal/petition/open/viewdetail/PRI80995be2cefd4f3fa4b9fb4b6f1204c8"

In [109]:
res_=requests.get(x)
soup_=bs(res_.text,"html.parser")